In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
from io import StringIO
import requests
import csv
import time
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
from selenium.webdriver.common.keys import Keys
from parsel import Selector

In [11]:
#make sure to change to your path on computer
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe')
driver.get("https://masslandlords.net/policy/eviction-data/filings-week-ending-2021-02-20/")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#hmtl for whole website
print(soup)

<html class="win chrome chrome8 webkit webkit5" lang="en-US" style=""><!--<![endif]--><head>
<meta charset="utf-8"/>
<!-- Google Tag Manager -->
<script async="" id="groove-iframe-shim-loader" src="https://006f5c34-3d62-4a7c-b55e-dd76f40feebb.widget.cluster.groovehq.com/api/shim/0676bd0ec17e599d0436449b6a43adfe" type="text/javascript"></script><script async="" crossorigin="anonymous" src="https://connect.facebook.net/en_US/all.js?hash=a8ef73d28348bbd6cedc11c962b5549e&amp;ua=modern_es6"></script><script async="" id="groove-script" src="https://006f5c34-3d62-4a7c-b55e-dd76f40feebb.widget.cluster.groovehq.com/api/loader" type="text/javascript"></script><script id="facebook-jssdk" src="//connect.facebook.net/en_US/all.js#xfbml=1&amp;appId=349176241838374"></script><script async="" src="//bat.bing.com/bat.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></script><script async="" src="https://www.googletagservices.com/tag/js/gpt.js" type="text/javascript"></script>

In [15]:
#get all the text within the website
names = driver.find_element_by_class_name('monospace').text
print(names)

Residential summary process: Filings Report

This report examines cases filed recently before reading, for which outcomes were largely unknown.

Search Period Start: 2021-02-14
Search Period End:   2021-02-20
Earliest Case:       2021-02-15
Latest Case:         2021-02-19
Total Cases:                423
Total Transfers:             23

--
High-level take-aways:
Percentage of landlords for whom attorney is optional: 25.5%
Of those, percentage pro se:                           55.6%

Most common cause:                        Non-payment
Least stable municipality/neighborhood: Chestnut Hill
Least stable with 10+ filings:             Fall River
Least stable with 100+ filings:           (not given)
Least stable county:                       Barnstable

Cases per day: 70
Housing crisis baseline max cases per day (UCL): 130
Housing crisis baseline min cases per day (LCL): 44
The number of filings this period is not statistically different from the pre-pandemic housing crisis baseline.
--

Cou

# The text is split by '--' we split into five sections

#split the text based on '--'
a,b,c,d,e,f =names.split('--')

In [23]:
#summary on dates and totals
print(a)

Residential summary process: Filings Report

This report examines cases filed recently before reading, for which outcomes were largely unknown.

Search Period Start: 2021-02-14
Search Period End:   2021-02-20
Earliest Case:       2021-02-15
Latest Case:         2021-02-19
Total Cases:                423
Total Transfers:             23




In [24]:
# more summary data
print(b)


High-level take-aways:
Percentage of landlords for whom attorney is optional: 25.5%
Of those, percentage pro se:                           55.6%

Most common cause:                        Non-payment
Least stable municipality/neighborhood: Chestnut Hill
Least stable with 10+ filings:             Fall River
Least stable with 100+ filings:           (not given)
Least stable county:                       Barnstable

Cases per day: 70
Housing crisis baseline max cases per day (UCL): 130
Housing crisis baseline min cases per day (LCL): 44
The number of filings this period is not statistically different from the pre-pandemic housing crisis baseline.



In [25]:
#convert to tables
print(c)



Courts
(n)                           Count   Percent
northeast                        88     20.8%
western                          76     18.0%
central                          67     15.8%
southeast                        56     13.2%
eastern                          51     12.1%
metro_south                      23      5.4%
chelsea district                 12      2.8%
cambridge district                9      2.1%
quincy district                   8      1.9%
somerville district               7      1.7%
fall river district               6      1.4%
pittsfield district               4      0.9%
new bedford district              3      0.7%
barnstable district               2      0.5%
dedham district                   2      0.5%
framingham district               2      0.5%
malden district                   2      0.5%
northern berkshire district       2      0.5%
palmer district                   2      0.5%
lynn district                     1      0.2%


Party Type
(n)         

In [26]:
#convert to tables
print(d)


Data Sources (report errors to hello@masslandlords.net):


County Data Households Percent Renters
Barnstable      96,509           20.0%
Berkshire       53,792           30.0%
Bristol        220,528           37.3%
Dukes           18,146           22.6%
Essex          297,898           36.9%
Franklin        30,927           34.1%
Hampden        179,970           39.5%
Hampshire       60,002           30.7%
Middlesex      612,366           38.1%
Nantucket       11,399           30.8%
Norfolk        269,717           31.6%
Plymouth       191,041           22.6%
Suffolk        309,945           63.7%
Worcester      316,162           35.2%

County Households:
2019 ACS TableID S1101

County Households and Renters, Dukes and Nantucket:
https://www.census.gov/quickfacts/dukescountymassachusetts

County Households and Renters, Large Counties:
https://data.census.gov/cedsci/table?q=S11&g=0400000US25.050000&d=ACS%201-Year%20Estimates%20Subject%20Tables&tid=ACSST1Y2019.S1101&hidePreview=true

Du

In [27]:
# methologies, worth reading but no need to convert to tables
print(e)


Methodology

Publicly available records at MassCourts.org are read fully manually at two intervals:

 1.) For a filings report, once within approximately one week of filing;

 2.) For an outcomes report, once again approximately 12 to 18 months after filing.

Cases are searched by 'case type' = 'summary process', party type = 'plaintiff' within date ranges, typically weekly.

MassCourts.org displays matches, not cases. X plaintiffs on a single case result in X separate matches. Cases are manually de-duplicated. Where the number of matches exceeds the number displayed (e.g., 'Displaying 100 of 257 total matches.'), date ranges are reduced until all cases may be viewed. If the date range cannot be reduced (i.e., is one day) but matches still exceed the display limit of 100, then two additional passes are taken. First, municipalities are filtered in stages. Second, to capture cases with no municipality entered, the search switches from 'case type' to 'case number', guessing the missing n

In [28]:
#credits (no need to convert to tables)
print(f)



When Citing This Work, Please Credit:

MassLandlords, Inc. Available online at https://masslandlords.net/policy/eviction-data/.


# SPLIT UP ALL TABLES THEN PRINT THEM

In [43]:
c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12 = c.split("\n\n\n")



In [45]:
print(c1)



Courts
(n)                           Count   Percent
northeast                        88     20.8%
western                          76     18.0%
central                          67     15.8%
southeast                        56     13.2%
eastern                          51     12.1%
metro_south                      23      5.4%
chelsea district                 12      2.8%
cambridge district                9      2.1%
quincy district                   8      1.9%
somerville district               7      1.7%
fall river district               6      1.4%
pittsfield district               4      0.9%
new bedford district              3      0.7%
barnstable district               2      0.5%
dedham district                   2      0.5%
framingham district               2      0.5%
malden district                   2      0.5%
northern berkshire district       2      0.5%
palmer district                   2      0.5%
lynn district                     1      0.2%


In [46]:
print(c2)

Party Type
(n)                Plaintiffs   Defendants
Corporate Entity          315            2
Natural Person            108          421
Total                     423          423


In [47]:
print(c3)

(%)                Plaintiffs   Defendants
Corporate Entity        74.5%         0.5%
Natural Person          25.5%        99.5%
Total                  100.0%       100.0%


In [48]:
print(c4)

Plaintiff Representation
(n)        Has Attorney   Pro Se   Total
Required            311        4     315
Optional             48       60     108
Total               359       64     423


In [49]:
print(c5)

(%)        Has Attorney   Pro Se   Total
Required          73.5%     0.9%   74.5%
Optional          11.3%    14.2%   25.5%
Total             84.9%    15.1%  100.0%


In [50]:
print(c6)

Defendant Representation
Important: Defendants will not have an attorney known to the plaintiff at time of filing unless this is an ongoing matter. 100% pro se is to be expected in all filing reports except to the extent time passes between filing and staff review. See outcomes report for final assessment.

(n)        Has Attorney   Pro Se   Total
Required              0        2       2
Optional             14      407     421
Total                14      409     423


In [51]:
print(c7)

(%)        Has Attorney   Pro Se   Total
Required           0.0%     0.5%    0.5%
Optional           3.3%    96.2%   99.5%
Total              3.3%    96.7%  100.0%


In [52]:
print(c8)

Number of Adults in Households   Count   Percent
0                                   38      9.0%
1                                  300     70.9%
2                                   72     17.0%
3                                    9      2.1%
4                                    1      0.2%
5                                    3      0.7%
Total                              423    100.0%
Note: Households may appear with zero adults due to clerical delay entering known defendants, identity protection obscuring known defendants (e.g., 42 USC Sections 13701 through 1404 Violence Against Women Act), or no adult defendants (e.g., abandonment of minors).


In [54]:
print(c9)


Initiating Action   Count   Percent
Non-payment           272     64.3%
Cause                  69     16.3%
No Fault               50     11.8%
Unknown/Other          30      7.1%
Foreclosure             2      0.5%


In [55]:
print(c10)

                  Rate per 100,000          
Municipality             Residents     Count
Chestnut Hill                  188         4
East Falmouth                   69         4
Pocasset                        34         1
Dennis Port                     31         1
Millville                       31         1
Rockland                        28         5
Chelsea                         25         9
Fall River                      24        22
Southbridge                     23         4
New Bedford                     18        18
West Yarmouth                   18         1
Norwood                         17         5
Revere                          15         8
Hyannis                         14         2
North Adams                     14         2
Whitman                         13         2
Abington                        12         2
Cambridge                       12        13
Hyde Park                       11         4
Allston                         10         3
Canton    

In [120]:
print(c11)


            Rate per 100,000          
County     Renter Households     Count
Barnstable                56        11
Plymouth                  55        24
Bristol                   54        45
Berkshire                 37         6
Norfolk                   28        24
Suffolk                   27        54
Worcester                 16        18
Middlesex                 11        27
Essex                      2         3
Dukes                      0         0
Franklin                   0         0
Hampden                    0         0
Hampshire                  0         0
Nantucket                  0         0
Rate per 100,000 renter households based on 2019 ACS. Counties appear even if no evictions filed.


# Convert Text Tables into Pandas Dataframes

In [143]:
data=StringIO(c1)
df=pd.read_csv(data)
df.columns=df.iloc[0]
df=df.iloc[1:]
df
#next step is to split into three columns

,(n) Count Percent
1,northeast 88 20.8%
2,western 76 18.0%
3,central 67 15.8%
4,southeast 56 13.2%
5,eastern 51 12.1%
6,metro_south 23 5.4%
7,chelsea district 12 2.8%
8,cambridge district 9 2.1%
9,quincy district 8 1.9%
10,somerville district 7 1.7%
